In [1]:
import os
import numpy as np
from utils.image import read_reorient_nifti
import nibabel as nib
from utils.analysis import compute_dice
import json

In [10]:
t = np.empty((0,3))
set(t[:,2])

set()